## Quantifiers and scope tutorial

### Kyle Rawlins  11/16

This notebook walks through several basic techniques for handling quantifier scope, together with how to implement them in the Lambda Notebook.

### Quantifiers in object position


In [ ]:
%%lamb
||every|| = L f_<e,t> : L g_<e,t> : Forall x_e : f(x) >> g(x)
||doctor|| = L x_e : Doctor(x)
||someone|| = L f_<e,t> : Exists x_e : Human(x) & f(x)
||saw|| = L x_e : L y_e : Saw(y,x)
||alfonso|| = Alfonso_e

In [ ]:
((every * doctor) * (saw * alfonso)).tree()

In [ ]:
(saw * (every * doctor))

### Quantifiers in object position via QR

The "standard" approach is to move an object position quantified DP so it scopes over its immediate TP.  The lambda notebook currently has no explicit syntax beyond order of composition, so this isn't done automatically for you (feel free to submit a pull request...).  However, it is easy to do "by hand".  We will need traces and binders.  A version of Predicate Abstraction (PA) is already present as a composition operation in the default system.

In [ ]:
trace = lang.Trace(index=2, typ=tp("e"))
trace

In [ ]:
binder = lang.Binder(index=2)
binder

In [ ]:
lang.get_system()

In [ ]:
(every * doctor) * (binder * (alfonso * (saw * trace)))

In [ ]:
((every * doctor) * (binder * (alfonso * (saw * trace)))).tree()

In [ ]:
(every * doctor) * (binder * (someone * (saw * trace)))

To get surface scope, you need to also move the subject (after moving the object).  This results in a second trace / binding operator.

In [ ]:
trace5 = lang.Trace(index=5, typ=tp("e"))
binder5 = lang.Binder(index=5)
(someone * (binder5 * ((every * doctor) * (binder * (trace5 * (saw * trace))))))

### Quantifiers in object position via type shifting

An alternative approach is to type-shift the DP to a type where it can take a transitive predicate and ignore the external argument position.

This would need to be generalized for n-ary predicates.

In [ ]:
%%lamb
||every|| = L f_<e,t> : L g_<e,t> : Forall x_e : f(x) >> g(x)
||doctor|| = L x_e : Doctor(x)
||someone|| = L f_<e,t> : Exists x_e : Human(x) & f(x)
||saw|| = L x_e : L y_e : Saw(y,x)
||alfonso|| = Alfonso_e

The following combinator shifts a GQ type into something that can handle a transitive verb.

In [ ]:
gq_lift_combinator = %te L f_<<e,t>,t> : L g_<e,<e,t>> : L x_e : f(L y_e : g(y)(x))
gq_lift_combinator

In [ ]:
gq_lift_combinator.type

In [ ]:
gq_lift_combinator(someone.content).reduce_all()

In [ ]:
system = lang.td_system.copy()
system.add_rule(lang.unary_factory(gq_lift_combinator, "gq-lift-trans", typeshift=True))
system.typeshift = True
lang.set_system(system)
system

In [ ]:
(alfonso * (saw * someone))

In [ ]:
(alfonso * (saw * someone)).tree()

In [ ]:
(someone * (saw * (every * doctor)))

In [ ]:
((every * doctor) * (saw * someone))

### Quantifier scope via type shifting

This so far produces only surface scope readings when there are multiple quantifiers.

*Approach 1*: Following work in CCG, one might imagine that composition needn't match constituency; if the subject shifts and composes with the verb before the object we can get the other scoping.  (In CCG this is implemented using a function composition operation, not a type-shift.)

*Approach 2*: Someone interested in constituency might find this unsatisfying.  How could this be resolved using a type-shift?  One idea (due to Hendriks) is to build scope-taking shifts that operate on verb meanings.

In [ ]:
surface_shift_comb = %te L v_<e,<e,t>> : L f_<<e,t>,t> : L g_<<e,t>,t> : g(L y_e : f(L x_e : (v(x)(y))))
inverse_shift_comb = %te L v_<e,<e,t>> : L f_<<e,t>,t> : L g_<<e,t>,t> : f(L x_e : g(L y_e : (v(x)(y))))

inverse_shift_comb(saw.content).reduce_all()

In [ ]:
surface_shift_comb(saw.content).reduce_all()

Let's create a new composition system with both of these combinators used as typeshifts.

In [ ]:
system = lang.td_system.copy()
system.add_rule(lang.unary_factory(surface_shift_comb, "surface", typeshift=True))
system.add_rule(lang.unary_factory(inverse_shift_comb, "inverse", typeshift=True))
system.typeshift = True
lang.set_system(system)
system

In [ ]:
r = (someone * ((every * doctor) * saw))
r

In [ ]:
r[0].tree()

In [ ]:
r[1].tree()

*Approach 3*: A final strategy would be to provide the first gq-shifter plus an even higher object type-lift that implements inverse scope.  This is effectively the combinator for Hendriks' inverse scope shifter with the order of arguments reversed.

In [ ]:
gq_lift_combinator = te("L f_<<e,t>,t> : L g_<e,<e,t>> : L x_e : f(L y_e : g(y)(x))")
gq_lift_combinator2 = te("L f_<<e,t>,t> : L g_<e,<e,t>> : L h_<<e,t>,t> : f(L y_e : h(L x_e : g(y)(x)))")

gq_lift_combinator2 #.type

In [ ]:
system = lang.td_system.copy()
system.add_rule(lang.unary_factory(gq_lift_combinator, "gq-lift-trans", typeshift=True))
system.add_rule(lang.unary_factory(gq_lift_combinator2, "gq-lift2-trans", typeshift=True))
system.typeshift = True
lang.set_system(system)
system

In [ ]:
r = (someone * ((every * doctor) * saw))
r

In [ ]:
r[1].tree()